In [2]:
import pandas as pd
import numpy as np
from faker import Faker
import random
from datetime import timedelta

fake = Faker()
Faker.seed(2026)

# --- 1. BẢNG DIM_PROJECTS ---
num_projects = 60
projects = [{
    'Project_ID': f"SMG_{i:03d}",
    'Project_Name': f"Project {fake.word().upper()}",
    'Client': fake.company(),
    'Service_Line': random.choice(['Digital', 'Branding', 'Production', 'Strategy']),
    'Region': random.choice(['APAC', 'EMEA', 'AMER'])
} for i in range(1, num_projects + 1)]
df_projects = pd.DataFrame(projects)

# --- 2. BẢNG DIM_STAFF_RATES ---
staff_rates = [
    {'Staff_Level': 'Director', 'Rate': 150},
    {'Staff_Level': 'Senior', 'Rate': 85},
    {'Staff_Level': 'Junior', 'Rate': 45},
    {'Staff_Level': 'Freelancer', 'Rate': 35},
    {'Staff_Level': 'Account', 'Rate': 70},
    {'Staff_Level': 'BD', 'Rate': 60}
]
df_staff = pd.DataFrame(staff_rates)

# --- 3. BẢNG FACT_PERFORMANCE (Tính P&L theo thời gian) ---
performance_data = []
stages = [
    ('1. Pitching', 0.0, 0.1, ['BD', 'Director']),
    ('2. Deposit', 0.3, 0.05, ['Account']),
    ('3. Strategy', 0.2, 0.25, ['Director', 'Senior']),
    ('4. Execution', 0.4, 0.5, ['Junior', 'Freelancer', 'Senior']),
    ('5. Liquidation', 0.1, 0.1, ['Account'])
]

for _, p in df_projects.iterrows():
    total_rev = random.randint(100000, 500000)
    total_hrs = (total_rev / 1000) * random.uniform(4, 7)
    start_date = fake.date_between(start_date='-1y', end_date='today')
    
    for idx, (name, rev_p, hrs_p, staffs) in enumerate(stages):
        stage_date = start_date + timedelta(days=idx*30)
        s_rev = total_rev * rev_p
        # Tính lương trung bình của team trong giai đoạn
        avg_r = np.mean([df_staff[df_staff['Staff_Level']==s]['Rate'].values[0] for s in staffs])
        s_labor = (total_hrs * hrs_p) * avg_r
        s_margin = s_rev - s_labor
        
        performance_data.append({
            'Project_ID': p['Project_ID'],
            'Date': stage_date,
            'Milestone': name,
            'Revenue': round(s_rev, 2),
            'Labor_Cost': round(s_labor, 2),
            'BD_Commission': round(s_margin * 0.05 if s_margin > 0 else 0, 2),
            'Account_Bonus': round(s_margin * 0.02 if s_margin > 0 else 0, 2),
            'Net_Profit': round(s_margin * 0.93 if s_margin > 0 else s_margin, 2),
            'Status': 'Completed' if stage_date < pd.Timestamp.today().date() else 'Planned'
        })

df_performance = pd.DataFrame(performance_data)

# Xuất file
df_projects.to_csv('Dim_Projects.csv', index=False)
df_staff.to_csv('Dim_Staff.csv', index=False)
df_performance.to_csv('Fact_Performance.csv', index=False)

In [3]:
# --- BẢNG 4: FACT_STAFF_ALLOCATION (Bảng cầu nối) ---
staff_alloc_data = []
for _, p in df_projects.iterrows():
    # Giả định mỗi dự án có một nhóm nhân sự cố định phụ trách
    # Phân bổ số giờ dự kiến cho từng cấp bậc
    allocs = [
        ('Director', random.randint(10, 30)),
        ('Senior', random.randint(40, 80)),
        ('Junior', random.randint(80, 150)),
        ('Account', random.randint(30, 60)),
        ('BD', random.randint(5, 15)),
        ('Freelancer', random.randint(0, 100) if p['Service_Line']=='Production' else 0)
    ]
    for level, hrs in allocs:
        staff_alloc_data.append({
            'Project_ID': p['Project_ID'],
            'Staff_Level': level,
            'Allocated_Hours': hrs
        })

df_staff_alloc = pd.DataFrame(staff_alloc_data)
df_staff_alloc.to_csv('Fact_Staff_Allocation.csv', index=False, encoding='utf-8-sig')